In [2]:
from pathlib import Path
import os,sys
import pickle
import pandas as pd

from tqdm.notebook import tqdm
import importlib

sys.path.insert(0, str(Path().resolve().parents[1]))
import fusemix.pipeline as pipeline 


In [3]:
def load_dataset(id):
    with open("../../test_data/fetched/dataset_"+str(id)+".pkl", "rb") as input:
        file = pickle.load(input)
    return file

In [4]:
ids = pd.read_csv("../../test_output/datasets_analysis.csv")['Id']

loaded_data = {}
for id in ids: loaded_data[id] = load_dataset(id = id)

In [5]:
n_runs = 10

md_param_grid = {
    'props': [0.75,1.],
    'mf_proportions': [0.5,0.75],
    'mnar_proportions': [0.,0.25,0.5]
}
"""
md_param_grid = {
    'props': [1.],
    'mf_proportions': [0.75],
    'mnar_proportions': [0.5]
}
"""

"\nmd_param_grid = {\n    'props': [1.],\n    'mf_proportions': [0.75],\n    'mnar_proportions': [0.5]\n}\n"

In [6]:
for id in ids:
    # create directory for dataset id in test_data/missing_data
    directory = "../../test_data/missing_data/"+str(id)
    if not os.path.exists(directory):
        os.makedirs(directory)
    # create directory for dataset id in test_data/missing_data/parameters
    for prop in md_param_grid['props']:
        for mf_proportion in md_param_grid['mf_proportions']:
            for mnar_proportion in md_param_grid['mnar_proportions']:
                directory = "../../test_data/missing_data/"+str(id)+"/"+str(prop)+"_"+str(mf_proportion)+"_"+str(mnar_proportion)
                if not os.path.exists(directory):
                    os.makedirs(directory)

### Generate amputations 

In [7]:
data_ = loaded_data[174]
print(data_['X_complete'].shape)
print(data_['y_complete'].shape)
data_['num_classes']

(195, 20)
(195, 1)


2

In [8]:
data_['X_complete']['MDVP_Jitter']

0      0.00784
1      0.00968
2      0.01050
3      0.00997
4      0.01284
        ...   
190    0.00459
191    0.00564
192    0.01360
193    0.00740
194    0.00567
Name: MDVP_Jitter, Length: 195, dtype: float64

In [9]:
for id in tqdm(ids):

    data_ = loaded_data[id]
    print(data_['X_complete'].shape)
    print(id)
    for prop in tqdm(md_param_grid['props']):
        for mf_proportion in tqdm(md_param_grid['mf_proportions']):
            for mnar_proportion in tqdm(md_param_grid['mnar_proportions']):
                
                directory = "../../test_data/missing_data/"+str(id)+"/"+str(prop)+"_"+str(mf_proportion)+"_"+str(mnar_proportion)
                
                for seed in tqdm(range(n_runs)):
                    print(seed)
                    cfg = pipeline.DataPipelineConfig(seed=seed,
                                                      prop=prop,
                                                      mnar_freq=mnar_proportion,
                                                      mf_proportion=mf_proportion,
                                                      complete_data=data_['X_complete'],
                                                      num_classes=data_['num_classes'],
                                                      verbose=False,
                                                      target=data_['y_complete'])
                    pipeline_ = pipeline.PipelineDataGeneration(cfg)
                    pipeline_.run()
                    
                    directory = "../../test_data/missing_data/"+str(id)+"/"+str(prop)+"_"+str(mf_proportion)+"_"+str(mnar_proportion)
                    with open(directory+"/data_pipeline_"+str(seed)+".pkl", 'wb') as f:  
                        pickle.dump(pipeline_, f)


  0%|          | 0/6 [00:00<?, ?it/s]

(195, 20)
174


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

...

5
6


2025-10-27 17:12:52,088 [WARNING] Candidates for pattern 7 all have almost the same weighted sum scores. It is possible this is due to the use of binary variables in amputation. This creates problems when using the sigmoid function for the score_to_probability_func. Currently our solution is as follows: if there is just one candidate with a sum score 0, we will ampute it. If there is one candidate with a nonzero sum score, or multiple candidates with the same score, we evenly apply as if MCAR.
2025-10-27 17:12:52,091 [WARNING] Candidates for pattern 8 all have almost the same weighted sum scores. It is possible this is due to the use of binary variables in amputation. This creates problems when using the sigmoid function for the score_to_probability_func. Currently our solution is as follows: if there is just one candidate with a sum score 0, we will ampute it. If there is one candidate with a nonzero sum score, or multiple candidates with the same score, we evenly apply as if MCAR.
20

7
8


2025-10-27 17:12:52,427 [WARNING] Candidates for pattern 9 all have almost the same weighted sum scores. It is possible this is due to the use of binary variables in amputation. This creates problems when using the sigmoid function for the score_to_probability_func. Currently our solution is as follows: if there is just one candidate with a sum score 0, we will ampute it. If there is one candidate with a nonzero sum score, or multiple candidates with the same score, we evenly apply as if MCAR.
2025-10-27 17:12:52,429 [WARNING] Candidates for pattern 10 all have almost the same weighted sum scores. It is possible this is due to the use of binary variables in amputation. This creates problems when using the sigmoid function for the score_to_probability_func. Currently our solution is as follows: if there is just one candidate with a sum score 0, we will ampute it. If there is one candidate with a nonzero sum score, or multiple candidates with the same score, we evenly apply as if MCAR.
2

9
